In [8]:
from langchain_community.document_loaders.csv_loader import CSVLoader
import sys
import csv
maxInt = sys.maxsize

while True:
    # decrease the maxInt value by factor 10 
    # as long as the OverflowError occurs.

    try:
        csv.field_size_limit(maxInt)
        break
    except OverflowError:
        maxInt = int(maxInt/10)

def encode_csv(path, chunk_size=1000, chunk_overlap=0):
    # Load PDF documents
    loader = CSVLoader(file_path=path,
        csv_args={
        'delimiter': ',',
        'quotechar': '"',
        'fieldnames': ['DR#', 'Problem Summary', 'Problem Description', 'Notes & Resolution']},
        metadata_columns=['DR#', 'Problem Summary', 'Problem Description', 'Notes & Resolution'],
        content_columns=['Problem Summary', 'Problem Description', 'Notes & Resolution'],
        encoding='utf-8')
    documents = loader.load()
    return documents

docs = encode_csv("data/mantis.csv")

# for doc in docs:
#     print(doc.metadata['Problem Summary'], doc.metadata['Problem Description'], doc.metadata['Notes & Resolution'])

In [9]:
from sentence_transformers import SentenceTransformer, CrossEncoder, util
from tqdm.autonotebook import tqdm
import torch
import ollama

if not torch.cuda.is_available():
    print("Warning: No GPU found. Please add GPU to your notebook")

#We use the Bi-Encoder to encode all passages, so that we can use it with semantic search
bi_encoder = SentenceTransformer('multi-qa-MiniLM-L6-cos-v1')
# bi_encoder.max_seq_length = 256     #Truncate long passages to 256 tokens
top_k = 5                          #Number of passages we want to retrieve with the bi-encoder

#The bi-encoder will retrieve 100 documents. We use a cross-encoder, to re-rank the results list to improve the quality
cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')

passages = []
for doc in docs:
    # print(doc.metadata['Problem Summary'], doc.metadata['Problem Description'], doc.metadata['Notes & Resolution'])
    passages.append(str([doc.metadata['Problem Summary'], doc.metadata['Problem Description'], doc.metadata['Notes & Resolution']]))

# We encode all passages into our vector space. This takes about 5 minutes (depends on your GPU speed)
corpus_embeddings = bi_encoder.encode(passages, convert_to_tensor=True, show_progress_bar=True)


Batches: 100%|██████████| 681/681 [00:09<00:00, 69.77it/s] 


In [10]:
# We also compare the results to lexical search (keyword search). Here, we use 
# the BM25 algorithm which is implemented in the rank_bm25 package.

from rank_bm25 import BM25Okapi
from sklearn.feature_extraction import _stop_words
import string
from tqdm.autonotebook import tqdm
import numpy as np


# We lower case our text and remove stop-words from indexing
def bm25_tokenizer(text):
    tokenized_doc = []
    for token in text.lower().split():
        token = token.strip(string.punctuation)

        if len(token) > 0 and token not in _stop_words.ENGLISH_STOP_WORDS:
            tokenized_doc.append(token)
    return tokenized_doc


tokenized_corpus = []
for passage in tqdm(passages):
    tokenized_corpus.append(bm25_tokenizer(passage))

bm25 = BM25Okapi(tokenized_corpus)


100%|██████████| 21792/21792 [00:00<00:00, 31779.83it/s]


In [11]:
# This function will search all wikipedia articles for passages that
# answer the query
def search(query):
    print("Input question:", query)

    ##### BM25 search (lexical search) #####
    # bm25_scores = bm25.get_scores(bm25_tokenizer(query))
    # top_n = np.argpartition(bm25_scores, -5)[-10:]
    # bm25_hits = [{'corpus_id': idx, 'score': bm25_scores[idx]} for idx in top_n]
    # bm25_hits = sorted(bm25_hits, key=lambda x: x['score'], reverse=True)
    
    # print("Top-N lexical search (BM25) hits")
    # for hit in bm25_hits[0:10]:
    #     print("\t{:.3f}\t{}".format(hit['score'], passages[hit['corpus_id']].replace("\n", " ")))

    ##### Semantic Search #####
    # Encode the query using the bi-encoder and find potentially relevant passages
    question_embedding = bi_encoder.encode(query, convert_to_tensor=True)
    question_embedding = question_embedding.cuda()
    hits = util.semantic_search(question_embedding, corpus_embeddings, top_k=top_k)
    hits = hits[0]  # Get the hits for the first query

    ##### Re-Ranking #####
    # Now, score all retrieved passages with the cross_encoder
    cross_inp = [[query, passages[hit['corpus_id']]] for hit in hits]
    cross_scores = cross_encoder.predict(cross_inp)

    # Sort results by the cross-encoder scores
    for idx in range(len(cross_scores)):
        hits[idx]['cross-score'] = cross_scores[idx]

    # Output of top-5 hits from bi-encoder
    print("\n-------------------------\n")
    print("Top-N Bi-Encoder Retrieval hits")
    hits = sorted(hits, key=lambda x: x['score'], reverse=True)
    for hit in hits[0:100]:
        print("\t{:.3f}\t{}".format(hit['score'], passages[hit['corpus_id']].replace("\n", " ")))

    # Output of top-5 hits from re-ranker
    print("\n-------------------------\n")
    print("Top-N Cross-Encoder Re-ranker hits")
    hits = sorted(hits, key=lambda x: x['cross-score'], reverse=True)
    result = list()
    for hit in hits[0:5]:
        print("\t{:.3f}\t{}".format(hit['cross-score'], passages[hit['corpus_id']].replace("\n", " ")))
        result.append(passages[hit['corpus_id']].replace("\n", " "))
    return(result)


In [12]:
# generate a response combining the prompt and data we retrieved in step 2
query = "I inserted a malfunction and the overcurrent trip did not occur when it was supposed to?"
data = search(query=query)
client = ollama.Client(host='http://localhost:11434')

model = "llama3.2:latest"
print(f"\n------------------------ Start {model} Response ------------------------ \n")
output = client.chat(model=model, messages=[{'role': 'user', 'content': f"Using this data: {data}. Respond to this prompt: {query}"}])
print(output.message.content)
print(f"\n------------------------- End {model} Response ------------------------- \n")

model = "phi4:latest"
print(f"\n------------------------ Start {model} Response ------------------------ \n")
output = client.chat(model=model, messages=[{'role': 'user', 'content': f"Using this data: {data}. Respond to this prompt: {query}"}])
print(output.message.content)
print(f"\n------------------------- End {model} Response ------------------------- \n")

model = "qwen2.5-coder:32b"
print(f"\n------------------------ Start {model} Response ------------------------ \n")
output = client.chat(model=model, messages=[{'role': 'user', 'content': f"Using this data: {data}. Respond to this prompt: {query}"}])
print(output.message.content)
print(f"\n------------------------- End {model} Response ------------------------- \n")

model = "llama3.3:latest"
print(f"\n------------------------ Start {model} Response ------------------------ \n")
output = client.chat(model=model, messages=[{'role': 'user', 'content': f"Using this data: {data}. Respond to this prompt: {query}"}])
print(output.message.content)
print(f"\n------------------------- End {model} Response ------------------------- \n")


Input question: I inserted a malfunction and the overcurrent trip did not occur when it was supposed to?

-------------------------

Top-N Bi-Encoder Retrieval hits
	0.612	['MCN601A-C condensate pump overcurrent trip', 'At rated power MCN601A-C causes overcurrent trip of breaker at about 70% severity.  Would it be possible to have the trip occur closer to 100% severity?', '@vhydro 11/19/2011\nVerified identified issue has been corrected.\n=-=\n@dweaver 11/17/2011\nIntegrated updated module EDSPreManIN.f onsite (11/17/11 4pm) into 9_CTI and 8_Exitech loads on simulator, and into 10_CTI_ASD load on DEV5 ASD workstation.\n=-=\n@jritter 11/17/2011\nTweaked the malfunction in EDSPreManin.f\n']
	0.565	['LPCS malfunction E21-051 pump overload', 'LPCS pmp overld alarm comes in when malfunction installed and pump not running.\r\nThis is incorrect', '@tpowell 09/26/2018\nRetest complete sat 9-26-18 t powell\n=-=\n@user340 09/25/2018\nThe ladder logic was calculating the e22_151x variable directl